# Complementaria Repaso Quiz 3 - Miércoles 3:30

In [2]:
import numpy as np
import pandas as pd
from jmarkov.mdp.dtmdp import dtmdp

In [3]:
# Matrices de transición (orden de estados: F, N, H)

P_S = np.array([
    [0.60, 0.30, 0.10],
    [0.10, 0.60, 0.30],
    [0.00, 0.30, 0.70]
])

P_B = np.array([
    [0.70, 0.25, 0.05],
    [0.20, 0.65, 0.15],
    [0.10, 0.60, 0.30]
])

P_V = np.array([
    [0.85, 0.15, 0.00],
    [0.40, 0.55, 0.05],
    [0.20, 0.60, 0.20]
])

P_R = np.array([
    [0.90, 0.10, 0.00],
    [0.50, 0.45, 0.05],
    [0.30, 0.60, 0.10]
])

# Costos térmicos por estado (USD por hora)
costos_termicos = {
    'F': 60,
    'N': 100,
    'H': 250
}

# Costos por acción (USD por hora)
costos_accion = {
    'S': 0,    # Sin ajuste
    'B': 40,   # Balanceo de carga
    'V': 70,   # Ventilación extra
    'R': 90    # Reducción de capacidad
}


## Parte A: Implementación del MDP

In [4]:
# Factor de descuento
beta = 0.95

# Espacio de estados y acciones
# F: Frío, N: Normal, H: Caliente
estados = np.array(['F', 'N', 'H'], dtype=str)

# S: Sin ajuste, B: Balanceo, V: Ventilación extra, R: Reducción de capacidad
acciones = np.array(['S', 'B', 'V', 'R'], dtype=str)

# Diccionario de matrices de transición por acción
matrices = {}
for a in acciones:
    if a == 'S':
        matrices[a] = P_S
    elif a == 'B':
        matrices[a] = P_B
    elif a == 'V':
        matrices[a] = P_V
    else:  # 'R'
        matrices[a] = P_R

# Matriz de retornos/costos R(s,a) = costo térmico + costo acción
# filas: estados [F, N, H], columnas: acciones [S, B, V, R]
retornos = np.zeros((len(estados), len(acciones)), dtype=float)

for i, s in enumerate(estados):
    for j, a in enumerate(acciones):
        retornos[i, j] = costos_termicos[s] + costos_accion[a]

retornos_df = pd.DataFrame(retornos, index=estados, columns=acciones)
retornos_df


,S,B,V,R
F,60.0,100.0,130.0,150.0
N,100.0,140.0,170.0,190.0
H,250.0,290.0,320.0,340.0


In [5]:
# Crear objeto MDP
mdp = dtmdp(estados, acciones, matrices, retornos, beta)
mdp

In [7]:
value_functions, optimal_policy = mdp.solve(0, minimize=True, method='value_iteration')

In [8]:
optimal_policy

{np.str_('F'): np.str_('S'),
 np.str_('N'): np.str_('V'),
 np.str_('H'): np.str_('R')}

In [9]:
value_functions

array([2509.73696485, 2634.34434529, 2825.25006827])

In [10]:
mdp.expected_policy_value(value_functions, optimal_policy)

np.float64(2588.2352939341995)

In [11]:
matrizPO = mdp.policy_transition_matrix(optimal_policy)

matrizPO_df = pd.DataFrame(matrizPO, index=estados, columns=estados)
matrizPO_df

,F,N,H
F,0.6,0.30,0.10
N,0.4,0.55,0.05
H,0.3,0.60,0.10


## Parte B: Simulación de Monte Carlo

In [ ]:
escenarios = 1000
horas = 24

np.random.seed(0)

# Matriz para guardar el costo de cada hora y escenario
costos = np.zeros((horas, escenarios), dtype=float)

# Matriz para marcar visitas al estado Frío (F) en las primeras 8 horas
indicador_F_prim8 = np.zeros(escenarios, dtype=int)

for j in range(escenarios):
    
    # Estado inicial: Normal (N)
    estado = 'N'
    
    for i in range(horas):
        # Acción según política óptima
        accion = str(optimal_policy[estado])
        
        # Costo inmediato en esa hora: costo térmico + costo acción
        costo_inmediato = costos_termicos[estado] + costos_accion[accion]
        costos[i, j] = costo_inmediato
        
        # Si estamos en las primeras 8 horas, marcar si se visita F
        if i < 8 and estado == 'F':
            indicador_F_prim8[j] = 1
        
        # Estado futuro según matriz de transición de la política óptima
        estado = np.random.choice(estados, p=matrizPO_df.loc[estado, :])


In [13]:
# Costo acumulado por escenario
costo_total_por_escenario = costos.sum(axis=0)

# Costo acumulado esperado (promedio sobre escenarios)
costo_esp_24h = costo_total_por_escenario.mean()
costo_esp_24h


np.float64(3139.5)

In [14]:
# Probabilidad estimada de visitar F al menos una vez en las primeras 8 horas
prob_visitar_F_prim8 = indicador_F_prim8.mean()
prob_visitar_F_prim8


np.float64(0.967)